## Anna Kaniowska - Cellphones & Accessories dataset analysis

The goal of this project is to extract as much information form the data set that can be obtained here - http://snap.stanford.edu/data/amazon/Cell_Phones_&_Accessories.txt.gz (source: http://snap.stanford.edu/data/web-Amazon-links.html)

In [1]:
# All imports needed to perform the analysis
import gzip
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#### Loading the dataset

In [ ]:
# A modified version of function available on the source page
def parse(filename):
    """
    Parses a gzipped text file and returns a list of dictionaries containing the parsed data.

    Parameters:
    filename (str): The path to the gzipped text file to be parsed.

    Returns:
    list: A list of dictionaries containing the parsed data.
    """
    f = gzip.open(filename, 'rb')
    entry = {}
    data = []
    for line in f:
        l = line.decode('utf-8').strip()
        colonPos = l.find(':')
        if colonPos == -1:
            data.append(entry)
            entry = {}
            continue
        eName = l[:colonPos]
        rest = l[colonPos+2:]
        entry[eName] = rest
    data.append(entry)
    return data

# Loading the dataset
data = parse('Cell_Phones_&_Accessories.txt.gz')
df = pd.DataFrame(data)


#### Getting to know the dataset

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
print(f"The shape of the dataset is: {df.shape}")

Taking a first look at the data, it is visible that it shows the reviews that customers gave to the products. The products are mainly cellphones and their accesories. The dataset is big - almost 79 000 rows is a significant amount of data. 10 columns provide information about the rated product, the customer and their opinion on the product. When it comes to technical details - it is necessary to change 'unknown' values to NaN in order to prepare data to further analysis. Checking the dataset for duplicated rows and dropping existing ones is also necessary because this is something that cannot be seen at first glance. Another conclusion is that the types of the columns are not necessarily correct (e.g. product/price should be stored as float), it is also needed to be corrected.

#### Checking for duplicated rows

In [ ]:
print(f"There is {df.duplicated().sum()} duplicated rows in the dataset")

In [ ]:
df.drop_duplicates(inplace=True)

#### Handling missing values

In [ ]:
df.replace("unknown", np.nan, inplace=True)

# Analyzing the missing values occurences
print("Missing values occurences:")
print(df.isna().sum())

# Checking anonymous reviews (those where userId and profileName is missing)
anon_reviews_perc = df['review/userId'].isna().sum()/df.shape[0] * 100
print(f"{anon_reviews_perc:.2f}% of the reviews are anonymously submitted.")


The first conclusion is one row that can be safely deleted in each column (it is very likely that it is the same row for each of the columns). \
The second conclusion refers to anonymous reviews. When missing values are less than 5% of given feature, they can be safely deleted without having impact on further analysis. For now the data will be divided into to sets - first with anonymous reviews and second with named reviews. It may be useful to further analysis.

In [ ]:
# Extracting columns with 1 missing value
cols = ['product/productId', 'product/title', 'review/helpfulness', 'review/score', 'review/time', \
        'review/summary', 'review/text']
df = df.dropna(subset=cols)
print(f"Shape of the dataset after dropping NaNs: {df.shape}")

As expected, only one row of the data was deleted.

Before dividing dataset into two separate dataset it would be useful to correct the columns' types.

#### Correcting columns types

In [ ]:
df['product/price'] = pd.to_numeric(df['product/price'], errors='coerce')
df['review/score'] = pd.to_numeric(df['review/score'], errors='coerce')
df['review/time'] = pd.to_datetime(df['review/time'].astype(float), unit='s')

def handle_helpfulness(x):
    """
    Converts the string representation of helpfulness scores to a float value between 0 and 1.

    Parameters:
        helpfulness (str): The string representation of helpfulness scores, in the format "x/y",
        where "x" is the number of users who found the review helpful and "y" is the total number of votes.

    Returns:
        float: The float value of the helpfulness score, calculated as "x / y". Returns 0 if "y" is 0.
    """
    try:
        nom, denom = x.split("/")
        return int(nom) / int(denom)
    except (ValueError, ZeroDivisionError):
        return 0

df['review/helpfulness'] = df['review/helpfulness'].apply(handle_helpfulness)

The price and score columns are stroing numeric values, the review has time in seconds. When it comes to helpfulness it was transformed to the float value that represents it.

#### Division of dataset

In [ ]:
# Dividing the dataset into anonymous and named
df_anon = df[(df['review/userId'].isna()) & (df['review/profileName'].isna())]
df_named = df[~(df['review/userId'].isna()) & ~(df['review/profileName'].isna())]

print(f"Missing values occurences in anonymous reviews dataset:\n{df_anon.isna().sum()}")
print(f"Missing values occurences in named reviews dataset:\n{df_named.isna().sum()}")

The last thing about missing values is the price. As it is a significant amount of data in both datasets it would not make sense to drop it. After seeing more details of the data a desicion will be made whether to replace the missing values with mean, mode or median or to create a new category for them.

#### Data Visualizations